In [ ]:
# This notebook reproduces the experiment with the parametric PDE in the paper.

In [ ]:
# the following setup lines need to be run the first time

!rm -r dolfin/
!sudo apt-get update
!sudo apt-get install -y --no-install-recommends fenics
!pip3 install fenics-ffc --upgrade
!git clone --branch=$"2019.1.0.post0" https://bitbucket.org/fenics-project/dolfin
!mkdir dolfin/build && cd dolfin/build && cmake .. && make install && cd ../..
!cd dolfin/python && pip3 install . && cd ../..

!apt update
!apt install -y cmake
!sudo apt-get install -y libatlas-base-dev
!sudo apt-get install -y gfortran
!sudo apt-get install -y libblas-dev liblapack-dev
!sudo apt install -y swig
!pip install pathlib

!git clone https://github.com/goroda/Compressed-Continuous-Computation.git c3


extract written to /work/dolfin/demo/documented/hyperelasticity/cpp/main.cpp
extract written to /work/dolfin/demo/documented/mixed-poisson/cpp/MixedPoisson.ufl
extract written to /work/dolfin/demo/documented/mixed-poisson/cpp/main.cpp
extract written to /work/dolfin/demo/documented/nonmatching-interpolation/cpp/P1.ufl
extract written to /work/dolfin/demo/documented/nonmatching-interpolation/cpp/P3.ufl
extract written to /work/dolfin/demo/documented/nonmatching-interpolation/cpp/main.cpp
extract written to /work/dolfin/demo/documented/poisson/cpp/Poisson.ufl
extract written to /work/dolfin/demo/documented/poisson/cpp/main.cpp
-- 
-- Generating form files in demo, test and bench directories. May take some time...
-- ----------------------------------------------------------------------------------------
-- 
-- Generating CMakeLists.txt files in demo, test and bench directories
-- -------------------------------------------------------------------
-- 
-- Copying demo and test data to buil

**modify c3py.py by hand (see the other notebook for details)**

In [ ]:
from dolfin import *

%cd /work/c3
!python setup.py build
!python setup.py install
import sys
sys.path.insert(1, '/root/venv/lib/python3.9/site-packages/c3py-0.0.7-py3.9-linux-x86_64.egg')

/work/c3
Add Extension  _c3_notsure
running build
running build_py
running build_ext
Creating Tempory File
Temporary file =  build/temp.linux-x86_64-3.9
cmake /work/c3 -DCMAKE_LIBRARY_OUTPUT_DIRECTORY=/work/c3/build/lib.linux-x86_64-3.9 -DCMAKE_BUILD_TYPE=Release -DBUILD_TESTS=OFF -DBUILD_EXAMPLES=OFF -DBUILD_PROFILING=OFF -DBUILD_BENCHMARKS=OFF -DBUILD_UTILITIES=OFF
-- A library with LAPACK API found.
-- Configuring done
-- Generating done
-- Build files have been written to: /work/c3/build/temp.linux-x86_64-3.9
cmake --build . --config Release -- -j4
[100%] Built target c3
Building Swig
['build/lib.linux-x86_64-3.9']
['$ORIGIN/']
Add Extension  _c3_notsure
running install
running bdist_egg
running egg_info
writing wrappers/python/c3py/c3py.egg-info/PKG-INFO
writing dependency_links to wrappers/python/c3py/c3py.egg-info/dependency_links.txt
writing requirements to wrappers/python/c3py/c3py.egg-info/requires.txt
writing top-level names to wrappers/python/c3py/c3py.egg-info/top_level.tx

In [ ]:
import c3py
import numpy as np
import scipy.io
from evalPDE import evalPDEfun
from time import time

global numevals
numevals = 0

def func2(xx,param=None):
    s = xx.shape[0]
    xx = xx*0.75+1.25;

    out = np.zeros(s);

    global mPDE
    for ii in range(s):
        out[ii] = evalPDEfun(xx[ii,:],mPDE)

    global numevals
    numevals = numevals + len(out)
    return out

In [ ]:
testPoints = scipy.io.loadmat('/work/testPoints.mat')
x_all = testPoints['x_all']
dlist_all = np.squeeze(testPoints['dlist_all'])

In [ ]:
# quick test that PDE can be solved
global mPDE 
mPDE = 4
global numevals 
numevals = 0

xx = np.zeros([1,mPDE*mPDE])
print(xx)
val = func2(xx,None)
print(val)
print(numevals)
numevals = 0

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Solving linear variational problem.
[0.03394709]
1


In [ ]:
testPoints = scipy.io.loadmat('/work/testPoints.mat')
x_all = testPoints['x_all']
dlist_all = np.squeeze(testPoints['dlist_all'])

tol = 1e-3 #1e-10
lb = -1                
ub = 1
cross_tol=tol
round_tol=tol
approx_tol=tol
verbose=0
adapt=0

init_rank = 2

tabledata = np.zeros([3,6])

for m in range(2,5):

    global mPDE 
    mPDE = m
    global numevals 
    numevals = 0

    d_index = np.where(dlist_all==m*m)[0][0]
    x = x_all[d_index][0]

    ftc = c3py.FunctionTrain(m*m)

    for jj in range(m*m):
        ftc.set_dim_opts(jj,"legendre",lb,ub,tol=approx_tol,coeff_check=4,nparam=14)

    t1 = time()
    ftc.build_approximation(func2,None,init_rank,verbose,adapt,cross_tol=cross_tol,round_tol=round_tol, maxiter=1)
    t2 = time()

    tabledata[m-2,1] = numevals

    yhat = ftc.eval(x)
    y = func2(x)
    # error, evals, dof, time, degree, ranks
    tabledata[m-2,0] = np.linalg.norm(y-yhat)/np.linalg.norm(y)
    tabledata[m-2,2] = ftc.get_nparam()
    tabledata[m-2,3] = t2-t1
    tabledata[m-2,4] = ftc.get_deg_mode().max()
    tabledata[m-2,5] = ftc.get_ranks().max()

print(tabledata)


Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational p

In [ ]:
np.save('/work/table.npy', tabledata)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=be8a243b-dea6-4fda-aa4e-c3eae4d12d22' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>